## Imports

In [16]:
# Read stocks
import yfinance as yf

# For plotting
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np
import pandas as pd

# To calculate TAs
import talib as ta
from talib import MA_Type

# For reading properties
from jproperties import Properties

## Load properties

In [17]:
configs = Properties()

with open('config/yf_ta_part2.properties', 'rb') as config_file:
     configs.load(config_file)

TICKER = configs.get('TICKER').data
START = configs.get('START').data
TEMPLATE  = configs.get('TEMPLATE').data

In [18]:
ticker = yf.Ticker(TICKER)
df = ticker.history(start=START)[['Open', 'Close', 'Volume', 'High', 'Low']]

# Create a Date column
df['Date'] = df.index
# Drop the Date as index
df.reset_index(drop=True, inplace=True)
df.head(5)

,Open,Close,Volume,High,Low,Date
0,217.609627,212.876648,37130100,218.069236,210.060323,2021-01-04 00:00:00-05:00
1,212.456152,213.082001,23823000,213.688302,210.930648,2021-01-05 00:00:00-05:00
2,207.478698,207.556931,35930700,211.703185,207.253787,2021-01-06 00:00:00-05:00
3,209.307336,213.463364,27694500,214.490150,208.984646,2021-01-07 00:00:00-05:00
4,213.844729,214.763947,22956200,215.702726,212.231218,2021-01-08 00:00:00-05:00


## Utility methods

In [19]:
# Sets padding for figures
def set_padding(fig):
    fig.update_layout(margin=go.layout.Margin(
        r=10, #right margin
        b=10)) #bottom margin

# Adds the range selector to given figure
def add_range_selector(fig):
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=[
                    dict(count=1, label='1m', step='month', stepmode='backward'),
                    dict(count=6, label='6m', step='month', stepmode='backward'),
                    dict(count=1, label='YTD', step='year', stepmode='todate'),
                    dict(count=1, label='1y', step='year', stepmode='backward'),
                    dict(step='all')
                ]),
            type='date'),#end xaxis  definition
        xaxis2_type='date')

# Adds the volume chart to row 2, column 1
def add_volume_chart(fig):
    # Colours for the Bar chart
    colors = ['#9C1F0B' if row['Open'] - row['Close'] >= 0
          else '#2B8308' for index, row in df.iterrows()]

    # Adds the volume as a bar chart
    fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], showlegend=False, marker_color=colors), row=2, col=1)

## Remove Empty Dates
Reference: __[A Simple Guide to Plotly for Plotting Financial Chart](https://plainenglish.io/blog/a-simple-guide-to-plotly-for-plotting-financial-chart-54986c996682)__

In [20]:
# removing all empty dates
# build complete timeline from start date to end date
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that are in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

## Plot CandleStick chart

In [21]:
fig = go.Figure(data=[go.Candlestick(x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])])

# Sets customized padding
set_padding(fig)

# Remove dates without values
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Update Y & X axis labels
fig.update_yaxes(title_text='Price')
fig.update_xaxes(title_text='Date')

fig.update_layout(title = TICKER + ' - CandleStick Chart', xaxis_rangeslider_visible=False, height=500, template=TEMPLATE)

## Plot RSI

In [22]:
# RSI
df['RSI'] = ta.RSI(df.Close,14)

# Construct a 2 x 1 Plotly figure
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.01, shared_xaxes=True)

# Plot RSI
fig.add_trace(go.Scatter(x=df['Date'], y=df['RSI'], name='RSI'), row=1, col=1)

# Plot Oversold and Overbought lines
fig.add_hline(y=30, line_dash='dash', line_color='limegreen', line_width=1)
fig.add_hline(y=70, line_dash='dash', line_color='red', line_width=1)
fig.update_yaxes(title_text='RSI Score')

# Add the volume chart
add_volume_chart(fig)

# Adds the range selector
add_range_selector(fig)

# Set the color from white to black on range selector buttons
fig.update_layout(xaxis=dict(rangeselector = dict(font = dict( color = 'black'))))

# Sets customized padding
set_padding(fig)

# Remove dates without values
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Update Y & X axis labels
fig['layout']['yaxis']['title']='RSI score'
fig['layout']['yaxis2']['title']='Volume'
fig['layout']['xaxis2']['title']='Date'

# Make it pretty and set the color from white to black on range selector buttons
layout = go.Layout(template=TEMPLATE, title=TICKER + ' - RSI', height=500, legend_title='Legend',
                   xaxis=dict(rangeselector = dict(font = dict( color = 'black'))))
fig.update_layout(layout)

## Plot Bollinger Bands

In [23]:
df['BU'], df['BM'], df['BL'] = ta.BBANDS(df.Close, timeperiod=20, matype=MA_Type.EMA)
fig = px.line(data_frame = df, x = df.index, y = ['Close', 'BU', 'BM', 'BL'])

# Update y & x axis labels
fig.update_yaxes(title_text='Price')
fig.update_xaxes(title_text='Date')
    
# Change the Close to Price for the lengend label
fig.data[0].name = 'Price'

# Sets customized padding
set_padding(fig)

# Make it pretty and set the color from white to black on range selector buttons
layout = go.Layout(template=TEMPLATE, title=TICKER + ' - Price, Bollinger Bands', height=500, legend_title='Legend')
fig.update_layout(layout)

## Add singals (buy and sell)
Reference:[Compute Bollinger Bands for stocks with Python and Pandas](https://tcoil.info/compute-bollinger-bands-for-stocks-with-python-and-pandas/)

In [24]:
# Drop Buy and Sell columns if they exist
df.drop(['Buy', 'Sell'], inplace=True, axis=1, errors='ignore')    

# Create buy and sell DataFrame
df_buy = df.query('Low < BL')[['Date', 'Close']]
df_sell = df.query('High > BU')[['Date', 'Close']]

# Round close values for both buy and sell
df_buy['Close'] = round(df_buy.Close.round())
df_sell['Close'] = round(df_sell.Close.round())

## Plot Buy and Sell signals

In [25]:
fig = go.Figure(data=[go.Candlestick(x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'],
    close=df['Close'], name='Candlestick')])

# Plot BU line graph; don't show legend
fig.add_trace(go.Scatter(x=df['Date'], y=df['BU'], fill=None, mode='lines', showlegend=False))

# Plot BL line graph and fill upto BU; don't show legend
fig.add_trace(go.Scatter(x=df['Date'], y=df['BL'], fill='tonexty', mode='lines', showlegend=False))

# Plot Buy signals
fig.add_trace(go.Scatter(x=df_buy['Date'], y=df_buy['Close'], mode='markers',
    marker=dict(symbol='x', size=7, line=dict(width=1)), name = 'Buy'))

# Plot Sell Signls
fig.add_trace(go.Scatter(x=df_sell['Date'], y=df_sell['Close'], mode='markers',
    marker=dict(symbol='diamond', size=7, line=dict(width=1)), name = 'Sell'))

# Update y  & x axis labels
fig.update_yaxes(title_text='Price')
fig.update_xaxes(title_text='Date')
    
# Change the Close to Price for the lengend label
fig.data[0].name = 'Price'

# Sets customized padding
set_padding(fig)

# Make it pretty
layout = go.Layout(template=TEMPLATE, title = TICKER + ' - Buy / Sell Signals', height=500,
    xaxis_rangeslider_visible=False
)
fig.update_layout(layout)

## Plot MACD

In [26]:
# Calculate MACD values
# Empty Data Frame to collect MACD analysis results
analysis = pd.DataFrame()
analysis['macd'], analysis['macdSignal'], analysis['macdHist'] = ta.MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)

# Construct a 2 x 1 Plotly figure
fig = make_subplots(rows=2, cols=1)

# Candlestick chart for pricing
fig.append_trace(go.Candlestick(x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'],
    close=df['Close'], showlegend=False), row=1, col=1)

# Fast Signal (%k)
fig.append_trace(go.Scatter(x=df['Date'], y=analysis['macd'], line=dict(color='#C42836', width=1),
    name='MACD Line'), row=2, col=1)

# Slow signal (%d)
fig.append_trace(go.Scatter(x=df['Date'], y=analysis['macdSignal'], line=dict(color='limegreen', width=1),
    name='Signal Line'), row=2, col=1)

# Colorize the histogram values
colors = np.where(analysis['macd'] < 0, '#EA071C', '#57F219')

# Plot the histogram
fig.append_trace(go.Bar(x=df['Date'], y=analysis['macdHist'], name='Histogram', marker_color=colors), row=2, col=1)

# Update y & x axis labels
fig['layout']['yaxis']['title']='Price'
fig['layout']['xaxis2']['title']='Date'

# Change the Close to Price for the lengend label
fig.data[0].name = 'Price'

# Sets customized padding
set_padding(fig)

# Make it pretty
layout = go.Layout(template=TEMPLATE, title = TICKER + ' - MACD Indicator', height=700,
    xaxis_rangeslider_visible=False)

# Update options and show plot
fig.update_layout(layout)

fig.update_layout(legend=dict(yanchor="top", y=0.45, xanchor="left", x=1.01))
fig.show()